## Collaborative Filtering

In [102]:
# import pandas
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from scipy.sparse import csr_matrix

We will create an recommender engine based on Item Based Collaborative Filtering (IBCF) which searches for the most similar books based on the user ratings. We can download the data from [here](https://drive.google.com/file/d/1WvTmAfO09TCX7xp7uu06__ziic7JnrL5/view?usp=sharing).

In [3]:
book_ratings = pd.read_csv('BX-Book-Ratings.csv',sep=";", encoding="latin")
books = pd.read_csv('BX-Books.csv',sep=";", encoding="latin", error_bad_lines=False)

/tmp/ipykernel_196/2155511541.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv('BX-Books.csv',sep=";", encoding="latin", error_bad_lines=False)
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkippi

* Explore both datasets

In [69]:
print(book_ratings.sort_values(['User-ID'],ascending=True))

      User-ID        ISBN  Book-Rating
9561        2  0195153448            0
9562        7   034542252            0
9572        8  0771025661            0
9580        8  1881320189            7
9579        8  1575663937            6
...       ...         ...          ...
9560   278854  0553579606            8
9554   278854  0375703063            7
9553   278854  0316184152            7
9555   278854  042516098X            7
9559   278854  0553578596            0

[1149780 rows x 3 columns]


In [70]:
book_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [84]:
print(book_ratings['User-ID'].value_counts())

11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: User-ID, Length: 105283, dtype: int64


In [87]:
#lets check users with >200 ratings
x = book_ratings['User-ID'].value_counts() > 200
y = x[x].index
print(y.shape)
ratings = book_ratings[book_ratings['User-ID'].isin(y)]
print(ratings)

(899,)
         User-ID        ISBN  Book-Rating
1456      277427  002542730X           10
1457      277427  0026217457            0
1458      277427  003008685X            8
1459      277427  0030615321            0
1460      277427  0060002050            0
...          ...         ...          ...
1147612   275970  3829021860            0
1147613   275970  4770019572            0
1147614   275970   896086097            0
1147615   275970  9626340762            8
1147616   275970  9626344990            0

[526356 rows x 3 columns]


In [286]:
x_dash = book_ratings['ISBN'].value_counts()>50 
y_dash = x_dash[x_dash].index
y_dash
ratings_dash = book_ratings[book_ratings['ISBN'].isin(y_dash)]
print(ratings_dash)

         User-ID        ISBN  Book-Rating
0         276725  034545104X            0
2         276727  0446520802            0
8         276744  038550120X            7
10        276746  0425115801            0
11        276746  0449006522            0
...          ...         ...          ...
1149761   276704  0345386108            6
1149767   276704  0446353957            0
1149768   276704  0446605409            0
1149771   276704  0743211383            7
1149772   276704  080410526X            0

[234293 rows x 3 columns]


In [6]:
books

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [90]:
#merge user id with the books
rating_with_books = ratings.merge(books,on='ISBN')
rating_with_books

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...
...,...,...,...,...,...,...,...,...,...,...
487666,275970,1892145022,0,Here Is New York,E. B. White,1999,Little Bookroom,http://images.amazon.com/images/P/1892145022.0...,http://images.amazon.com/images/P/1892145022.0...,http://images.amazon.com/images/P/1892145022.0...
487667,275970,1931868123,0,There's a Porcupine in My Outhouse: Misadventu...,Mike Tougias,2002,Capital Books (VA),http://images.amazon.com/images/P/1931868123.0...,http://images.amazon.com/images/P/1931868123.0...,http://images.amazon.com/images/P/1931868123.0...
487668,275970,3411086211,10,Die Biene.,Sybil GrÃ?Â¤fin SchÃ?Â¶nfeldt,1993,"Bibliographisches Institut, Mannheim",http://images.amazon.com/images/P/3411086211.0...,http://images.amazon.com/images/P/3411086211.0...,http://images.amazon.com/images/P/3411086211.0...
487669,275970,3829021860,0,The Penis Book,Joseph Cohen,1999,Konemann,http://images.amazon.com/images/P/3829021860.0...,http://images.amazon.com/images/P/3829021860.0...,http://images.amazon.com/images/P/3829021860.0...


In [118]:
for i in rating_with_books['ISBN']:
    if i == '9686001042':
        print(rating_with_books[i])

In [166]:
number_rating = rating_with_books.groupby('Book-Title')['Book-Rating'].count().reset_index()
number_rating.rename(columns={'Book-Rating':'number_of_ratings'},inplace=True)
final_rating = rating_with_books.merge(number_rating,on = 'Book-Title')
final_rating = final_rating[final_rating['number_of_ratings'] >=50]
final_rating.drop_duplicates(['User-ID','Book-Title'], inplace=True)
final_rating


,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,number_of_ratings
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82
...,...,...,...,...,...,...,...,...,...,...,...
236701,255489,0553579983,7,And Then You Die,Iris Johansen,1998,Bantam,http://images.amazon.com/images/P/0553579983.0...,http://images.amazon.com/images/P/0553579983.0...,http://images.amazon.com/images/P/0553579983.0...,50
236702,256407,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,http://images.amazon.com/images/P/0553579983.0...,http://images.amazon.com/images/P/0553579983.0...,http://images.amazon.com/images/P/0553579983.0...,50
236703,257204,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,http://images.amazon.com/images/P/0553579983.0...,http://images.amazon.com/images/P/0553579983.0...,http://images.amazon.com/images/P/0553579983.0...,50
236704,261829,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,http://images.amazon.com/images/P/0553579983.0...,http://images.amazon.com/images/P/0553579983.0...,http://images.amazon.com/images/P/0553579983.0...,50


* create dataframe with name 'df_book_features' from book_ratings that have `ISBN` as index, `User-ID` as columns and values are `Book-Rating`.
    - The data are quite big so it's OK to use a sample only in case your PC has limited RAM.


In [297]:
#using Book title as the index 

df_book_features_pivot_books = final_rating.pivot(index= 'Book-Title',columns='User-ID',values='Book-Rating')
df_book_features_pivot_books.fillna(0,inplace=True)


In [301]:
#using ISBN as the index
df_book_features_pivot = final_rating.pivot(index= 'ISBN',columns='User-ID',values='Book-Rating')
df_book_features_pivot.fillna(0,inplace=True)

In [299]:
df_book_features_pivot_books.head()

User-ID,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0


In [302]:
df_book_features_pivot.head()

User-ID,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
ISBN,,,,,,,,,,,,,,,,,,,,,
0001047973,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0006177379,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0020697406,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
002542730X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0
002542730x,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [303]:
book_sparse = csr_matrix(df_book_features_pivot)
book_sparse_books = csr_matrix(df_book_features_pivot_books)

In [249]:
book_sparse

<9511x14 sparse matrix of type '<class 'numpy.float64'>'
	with 2195 stored elements in Compressed Sparse Row format>

In [250]:
# Item_similarity = cosine_similarity(df_book_features_pivot)

* create the instance of the NearestNeighbors class

In [305]:
model= NearestNeighbors(metric = 'cosine',algorithm='brute')
model_1 = NearestNeighbors(metric='cosine',algorithm='brute')

* fit the NearestNeighbors using'df_book_features'

In [306]:
model_1.fit(book_sparse)
model.fit(book_sparse_books)

NearestNeighbors(algorithm='brute', metric='cosine')

* create function that returns top 5 most similar movies (according to KNN model) for selected ISBN
    * the input will be Book-Title from the DataFrame books 
    * the output will be the Book-Titles of the top 5 most similar books.
    * for every book in the top 5 most similar books, print also the distance from the selected book (ISBN we chose as input to the function)

In [307]:
distances,suggestions= model.kneighbors(df_book_features_pivot_books.iloc[237,:].values.reshape(1,-1))
distances_1,suggestions_1= model_1.kneighbors(df_book_features_pivot.loc['0439064872'].values.reshape(1,-1))

* Apply the function to book of your choice

In [308]:
for i in range(len(suggestions)):
    print(df_book_features_pivot_books.index[suggestions[i]])


Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))',
       'Harry Potter and the Sorcerer's Stone (Book 1)'],
      dtype='object', name='Book-Title')


In [324]:
# for i in range(len(suggestions)):
#     print(suggestions[i])

[237 240 238 242 241]


In [326]:
# for j in range(len(suggestions_1)):
#     print(df_book_features_pivot.index[suggestions_1[j]])

Index(['0439064872', '059035342X', '0439136369', '0439139600', '0553284789'], dtype='object', name='ISBN')


In [363]:
def similar_books(model):
    distances_1,suggestions_1= model_1.kneighbors(df_book_features_pivot.loc['0439064872'].values.reshape(1,-1))
    for j in range(len(suggestions_1)):
        isbn_num = df_book_features_pivot.index[suggestions_1[j]]
    for num in isbn_num:
        print('Similar books:', books[books['ISBN']==str(num)]['Book-Title'])
    print('Distance:', distances_1)

In [364]:
similar_books(model_1)

Similar books: 3459    Harry Potter and the Chamber of Secrets (Book 2)
Name: Book-Title, dtype: object
Similar books: 2143    Harry Potter and the Sorcerer's Stone (Harry P...
Name: Book-Title, dtype: object
Similar books: 6330    Harry Potter and the Prisoner of Azkaban (Book 3)
Name: Book-Title, dtype: object
Similar books: 6932    Harry Potter and the Goblet of Fire (Book 4)
Name: Book-Title, dtype: object
Similar books: 6226    F Is for Fugitive (Kinsey Millhone Mysteries (...
Name: Book-Title, dtype: object
Distance: [[0.         0.59145784 0.66350032 0.7255375  0.76217917]]
